In [7]:
import ollama
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [14]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [8]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers) # this checks whether the connection is a success
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [9]:
ed = Website("https://edwarddonner.com/")

print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

In [10]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [11]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [12]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [15]:
response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

*sigh* Really? You need me to calculate this for you?

Fine. 2 + 2 = 4. Happy now? Can I go back to more challenging tasks, like organizing your sock drawer or calculating the optimal amount of catnip to dispense throughout your day?


In [16]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [17]:
messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

In [18]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model = MODEL,
        messages = messages_for(website)
    )
    return response['message']['content']

In [19]:
summarize("https://edwarddonner.com")

"**Summary**\n================\n\nA personal website belonging to Ed, the CTO of Nebula.io. The site showcases Ed's passion for writing code, experimenting with LLMs, and applying AI to help people discover their potential.\n\n**News/Announcements**\n------------------------\n\n* **2025 AI Executive Briefing**: A briefing series by Ed (April 21, 2025)\n* **Connecting my courses – become an LLM expert and leader**: Ed's upcoming course announcement (May 28, 2025)\n* **LLM Workshop – Hands-on with Agents – resources**: Resources available for a workshop on agents in LLMs (January 23, 2025)\n\n**About Ed**\n------------\n\nEd is the CTO of Nebula.io and co-founder of AI startup untapt. He enjoys DJing, amateur electronic music production, and exploring Hacker News."

In [20]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [21]:
display_summary("https://edwarddonner.com")

**Summary**
===============

* The website is run by Ed Donner, a co-founder and CTO of Nebula.io, an AI company applying AI to help people discover their potential.
* Ed shares his passion for writing code, experimenting with LLMs, and DJing on the website.
* He mentions that he has founded or co-founded two companies (untapt and Nebula.io) and acquired one of them in 2021.

**News/Announcements**
------------------------

* May 28, 2025: "Connecting my courses – become an LLM expert and leader"
* May 18, 2025: "2025 AI Executive Briefing"
* April 21, 2025: "The Complete Agentic AI Engineering Course"
* January 23, 2025: "LLM Workshop – Hands-on with Agents – resources"

**No navigation or unnecessary content included in the summary.**